<center>
<a href="http://www.enac.fr/" ><img src="https://www.enac.fr/themes/custom/enac/assets/images/logo_1.png" style="float:left; max-width: 120px; display: inline" alt="ENAC"/></a> 
</center>

<font size="14">Manipulations et préparations de données avec </font>  <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 150px; display: inline" alt="Python"/></a> et <a href="http://pandas.pydata.org/"><img src="https://pandas.pydata.org/static/img/pandas.svg" style="max-width: 250px; display: inline" alt="Pandas"/></a>


**[Science des Données](https://e-campus.enac.fr/moodle/course/view.php?id=5684), Laurent Lapasset**, *2024*

Votre notebook s'execute actuellement sur :

In [1]:
import socket
print(socket.gethostname())

mds6


<hr>

**<span style="color:red">Pré-requis pour utiliser ce notebook</span>** :

Nous recommandons l'installation d'Ubuntu 20.04 LTS, système pour lequel tous les développements sont réalisés en premier par les GAFA, avant portage sur les autres OS.

Ce notebook fonctionne avec python 3.7 sous Windows et Linux (même si ce dernier est recommandé donc).
Il ne nécessite pas de ressources matérielles particulières.

**Avant** d'installer python et les librairies, il est recommandé de travailler dans votre environnement Anaconda spécifique. Pour cela vous devez créer cet environnement avec la commande `conda create --name testenv1` où `testenv1` sera votre nouvel environnement. On peut créer autant d'environnements que l'on souhaite dans la limite de l'espace disque disponible. Ensuite pour activer cet environnement `testenv1` et "rentrer" dedans utiliser la commande `source activate testenv1` et installer ce que vous souhaitez, ce que nous allons voir ci-dessous. Pour avoir la liste des environnements existants utilser la comande `conda env list`. Enfin, l'ensemble des options disponibles est accessibles avec `conda --help`. 

On suppose maintenant que vous avez créé et activé un environnement Conda.

Un certain nombre de librairies sont nécessaires pour le bon fonctionnement de ce notebook. Pour s'assurer d'une gestion cohérente et automatique des dépendances entre librairies, il est recommandé d'utiliser une des méthodes suivantes :

- via l'interface graphique d'anaconda : `anaconda-navigator`
- en ligne de commande : `conda install <librairie>`
- en ligne de commande : `conda install -c conda-forge <librairie>`
- en ligne de commande : `pip install <librairie>`

Remarques : 

- en salle de TP ENAC il faut passer à travers le proxy en utilisant :
    - l'option --proxy=http://10.222.118.213:3128 pour la commande pip.
    - positionner les variables HTTP_PROXY et HTTPS_PROXY dans le .bashrc avec la même valeur que pour pip.
- l'option `-c` indique le canal spécifique à partir duquel les packages peuvent être installés. `Conda` et `conda-forge` sont tous deux des gestionnaires de packages Python. `conda` est le gestionnaire de packages multiplateforme et `conda-forge` le canal de packages. Il y a quatre raisons principales d'utiliser le `conda-forge` canal au lieu du canal `defaults` maintenu par Anaconda:

    - Les packages sur `conda-forge` peuvent être plus à jour que ceux sur la chaîne `defaults`<br>
    - Certains packages sur la chaîne `conda-forge`  ne sont pas disponibles à partir de `defaults`<br>
    - Vous préférez utiliser une dépendance telle que `openblas` (de conda-forge) au lieu de `mkl` (de defaults).<br>
    - Si vous installez un package qui nécessite une bibliothèque compilée (par exemple, une extension C ou un wrapper autour d'une bibliothèque C), cela peut réduire le risque d'incompatibilités si vous installez tous les packages dans un environnement à partir d'un seul canal en raison de la compatibilité binaire de la bibliothèque C (mais cet avis peut être obsolète / changer dans le futur).<br>

- sur la dernière version d'Ubuntu 20.04 LTS c'est python 3 qui est installé par défaut. A partir de cette release il n'ai plus nécessaire d'utiliser pip3 à la place de pip.
<hr>

**Résumé**:  

Nous allons utiliser Python pour la préparation (*data munging* ou *wrangling*) de données qui tiennent en mémoire une fois réorganisées. Nous allons aborder les fonctionnalités de la librairie `pandas` et à la classe `DataFrame` : lire et écrire des fichiers ; apprendre à gérer une table de données et les types des variables ; échantillonner ; discrétiser ; regrouper des modalités ; faire des descriptions élémentaires uni et bi-variées ; réaliser une concaténation et jointure de tables.

La dernière partie optionnelle, illustre la mise en oeuvre de `Pandas` sur le problème du Titanic et c'est l'occasion d'appliquer et de compléter la visualisation de données exposée dans l'atelier dédié `Atelier-DataViz-Graph.ipyn`, en utilisant une nouvelle librairie [`Graphviz`](https://graphviz.org/), en calculant les courbes ROC et AUC, et la matrice de confusion. Cette dernière partie fait appel à des méthodes qui sont enseignées tout au long de la formation, aussi elle ne peut être correctement réalisée qu'en fin de cursus.

# Introduction

## Objectifs

Le *data munging* ou *wrangling* de données est l'ensemble des opérations permettant de passer de données brutes à une table (*data frame*) correcte et adaptée aux objectifs à atteindre par des méthodes statistiques d'analyse, d'exploration, de modélisation ou d'apprentissage. Nous utiliserons souvent les termes *manipulations* et *transformations* dans le cadre de ce notebook.

En présence de données complexes, peu ou mal organisées, présentant des trous, trop massives pour tenir en mémoire...  la qualité de cette étape est fondamentale (*garbage in garbage out*) pour la bonne réalisation d'une étude. Compte tenu de la diversité des situations envisageables, il serait vain de vouloir exposer tous les outils et techniques qui peuvent s'avérer nécessaires. Tâchons néanmoins de résumer les problèmes qui peuvent être rencontrés.

## Croissance du volume des données

Le volume des données et sa croissance occasionnent schématiquement trois situations :

1. Le fichier initial des données brutes peut être chargé *intégralement* en mémoire moyennant éventuellement de sauter quelques colonnes ou lignes du fichier (cf. section  3.1). C'est la situation la plus courante, tout logiciel statistique comme R peut réaliser les traitements. *C'est l'objet des prochaines sections de ce notebooks.
2. Le fichier initial est très volumineux mais la table (*DataFrame*), qui résulte de quelques manipulations (*munging*) appropriés, tient en mémoire. Cette situation nécessite : lecture, analyse, transformation, ré-écritures séquentielles du fichier ligne à ligne ou par bloc. Il existe des astuces avec R mais il est préférable d'utiliser des outils plus adaptés. Tout langage de programmation (java, c, perl, ruby...) peut être utilisé pour écrire le ou les programmes réalisant ce travail. Néanmoins Python, et plus précisément la librairie [`pandas`](http://pandas.pydata.org/), offre un ensemble d'outils efficaces pour accomplir ces tâches sans avoir à ré-inventer la roue et ré-écrire tout un ensemble de fonctionnalités relativement basiques. Remarque : les procédures `univariate` et `freq` et l'étape `data` de Statistical Analysis System (SAS) sont adaptées car elles ne chargent pas les données en mémoire pour réaliser des traitements simples. Néanmoins pour tout un tas de raisons trop longues à exposer, notamment de coût annuel de location, SAS perd régulièrement des parts de marché sur ce créneau. *Cette approche est introduite ci-dessous et consiste à enchâsser dans une même structure itérative et séquentielle les étapes précédentes.
3. Lorsque les données, très massives trop pour être entièrement chargées en mémoire, sont donc archivées sur un système de données distribuées (*Hadoop Distributed File System* ou HDFS par exmple que nous verrons en cours), ces manipulations et prétraitements doivent tenir compte alors de cet environnement : *Spark* et l'API `PySpark` permettant de gérer en python des données distribuées est à favoriser. *Cf. les [cours Hadoop et Spark](https://e-campus.enac.fr/moodle/course/view.php?id=15036#section-11)* concernés. 

### Quelques problèmes

Liste non exhaustive des problèmes pouvant être rencontrés et dont la résolution nécessite simultanément des compétences en Informatique, Statistique, Mathématiques et aussi "métier" du domaine de l'étude : 

-  Identifier les "individus" $\times$ "variables" (*instances*$\times$*features* en langue informatique) de la table à mettre en forme à partir de bases de données variées; *i.e.* logs d'un site web, listes d'incidents, localisations...
- Données atypiques (*outliers*): correction, suppression, transformation des variables ou méthode statistique robuste
- Variable qualitative avec beaucoup de modalités dont certaines très peu fréquentes : suppression, modalité `autres`, recodage aléatoire, regroupement "métier" ou méthode tolérante
- Distributions anormales (log-normale, Poisson, multimodales...) et problèmes d'hétéroscédasticité : transformation, discrétisation ou méthodes tolérantes
- Données manquantes: suppressions (ligne ou colonne), imputation ou méthodes tolérantes
- Représentations (splines, Fourier, ondelettes) et recalage (*time warping*) de données fonctionnelles.
- Représentation de trajectoires, de chemins sur un graphe
- Choix d'une distance (quadratique, absolue, géodésique...) entre les objets étudiés.
- ...

Bien entendu les "bons" choix dépendent directement de l'objectif poursuivi et des méthodes mises en oeuvre par la suite. D'où l'importance d'intégrer de façon précoce, dès la planification du recueil des données, les compétences statistiques nécessaires au sein d'une équipe.

### Fonctionnalités de `pandas`

La richesse des fonctionnalités de la librairie `pandas` est une des raisons, si ce n'est la principale,  d'utiliser Python pour extraire, préparer, éventuellement analyser, des données. En voici un bref aperçu. 
- *Objets* : les classes `Series` et `DataFrame` ou *table de données*.
- *Lire, écrire* création et exportation de tables de données à partir de fichiers textes (séparateurs, `.csv`, format fixe, compressés), binaires (HDF5 avec `Pytable`), HTML, XML, JSON, MongoDB, SQL... 
- *Gestion* d'une table : sélection des lignes, colonnes, transformations, réorganisation par niveau d'un facteur, discrétisation de variables quantitatives, exclusion ou imputation élémentaire de données manquantes, permutation et échantillonnage aléatoire, variables indicatrices, chaînes de caractères...
- *Statistiques* élémentaires uni et bivariées, tri à plat (nombre de modalités, de valeurs nulles, de valeurs manquantes...), graphiques associés, statistiques par groupe, détection élémentaire de valeurs atypiques...
- *Manipulation* de tables : concaténations, fusions, jointures, tri, gestion des types et formats...

### Références

Je remercie Brendan Guillouet et Philippe Besse de l'INSA Toulouse pour leur [notebook](https://github.com/wikistat) dont cet atelier s'inspire, qui lui-même s'inpire fortement du livre de référence (Mc Kinney, 2013) et de la [documentation en ligne](http://pandas.pydata.org/pandas-docs/stable/) à consulter sans modération. Cette documentation inclut également des [tutoriels](http://pandas.pydata.org/pandas-docs/stable/tutorials.html) à exécuter pour compléter et approfondir cette première initiation, sur un sujet relativement technique, et qui peut prendre des tournures très diverses en fonction de la qualité et des types de données traitées.

# Les classes `Series` et `DataFrame`
De même que la librairie `Numpy` introduit le type `array` indispensable à la manipulation de matrices en calcul scientifique, celle `pandas` introduit les classes `Series` (séries chronologiques) et  `DataFrame` ou table de données indispensables en statistique. 

## *Series*
La classe `Series` est l'association de deux `arrays` unidimensionnels. Le premier est un ensemble de valeurs indexées par le 2ème qui est souvent une série temporelle.  Ce type est introduit principalement pour des applications en Econométrie et Finance où Python est largement utilisé.

## *DataFrame*
Cette classe est proche de celle du même nom dans le langage R, il s'agit d'associer avec le même index de lignes des colonnes ou variables de types différents (entier, réel, booléen, caractère). C'est un tableau bi-dimensionnel avec des index de lignes et de colonnes mais il peut également être vu comme une liste de `Series` partageant le même index. L'index de colonne (noms des variables) est un objet de type `dict` (dictionnaire). C'est la classe qui sera principalement utilisée dans ce tutoriel. 

In [4]:
# Exemple de data frame
import pandas as pd
data = {"state": ["Ohio", "Ohio", "Ohio", 
       "Nevada", "Nevada"],
     "year": [2000, 2001, 2002, 2001, 2002],
     "pop": [1.5, 1.7, 3.6, 2.4, 2.9]}
frame = pd.DataFrame(data)
# ordre des colonnes
pd.DataFrame(data, columns=["year", "state", "pop"])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


In [9]:
# index des lignes et valeurs manquantes (NaN)
frame2=pd.DataFrame(data, columns=["year", "state", "pop", "debt"],
     index=["one", "two", "three", "four", "five"])
# liste des colonnes
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


In [10]:
print("coucou")

coucou


In [11]:
a=4

Ceci est du test 

In [12]:
b=2

In [13]:
c=a+b
print(c)

6


In [16]:
# Les deux première valeurs d'une colonne
frame["state"][0:2]

0    Ohio
1    Ohio
Name: state, dtype: object

In [ ]:
# toutes les valeurs d'une colonne
frame["state"]

In [17]:
frame.year

0    2000
1    2001
2    2002
3    2001
4    2002
Name: year, dtype: int64

In [ ]:
# "imputation"
frame2["debt"] = 16.5
frame2

In [18]:
# créer une  variable
frame2["eastern"] = frame2.state == "Ohio"
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,NaN,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,NaN,False
five,2002,Nevada,2.9,NaN,False


In [19]:
frame2.columns

Index(['year', 'state', 'pop', 'debt', 'eastern'], dtype='object')

In [20]:
# supprimer une  variable
del frame2[u"eastern"]
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

## Index
Les index peuvent être définis par emboîtement et beaucoup d'autres fonctionnalités sur la gestion des index sont décrites par Mac Kinney (2013) (chapitre 5) : 
- `append` nouvel index par concaténation,
- `diff` différence ensembliste,
- `intersection` intersection ensembliste,
- `union` union ensembliste
- `isin` vrai si la valeur est dans la liste,
- `delete` suppression de l'index $i$,
- `drop` suppression d'une valeur d'index, 
- `is_monotonic` vrai si les valeurs sont croissantes, 
- `is_unique` vrai si toutes les valeurs sont différentes, 
- `nique` tableau des valeurs uniques de l'index.

# Lire écrire des tables de données
`Pandas` offre des outils efficaces pour lire écrire des fichiers selon différents formats (csv, texte, fixe, compressé, xml, html, hdf5) ou interagir avec des bases de données SQL, MongoDB, des APIs web. Ce document se contente de décrire les fonctions les plus utiles `read_csv` et `read_table` pour lire des fichiers textes et générer un objet de classe ` DataFrame`. 

En principe ces fonctions font appel à un code écrit en C dont très rapide à l'exécution sauf pour l'emploi de certaines options (`skip\_footer, sep`} autre qu'un seul caractère), à éviter, qui provoquent une exécution en Python (`engine=Python`). 

La réciproque pour l'écriture est obtenue par les commandes `data.to_csv` ou `_table` avec des options similaires. 

## Syntaxe
L'exemple de base est donné pour lire un fichier au format `.csv` dont les valeurs sont séparées par des "," et dont la première ligne contient le nom des variables.
``
import pandas as pd
data=pd.read_csv("fichier.csv")
data=pd.read_table("fichier.csv", sep=",")
``

Il est important de connaître la liste des possibilités et options offertes par cette simple commande. Voici les principales ci-dessous et un lien à la [liste complète](http://pandas.pydata.org/pandas-docs/stable/io.html#io-read-csv-table).
- `path` chemin ou non du fichier ou URL.
- `sep` délimiteur comme \verb+ , ; | \t + ou \verb# \s+ #  pour un nombre variable d'espaces. 
- `header` défaut 0, la première ligne contient le nom des variables; si `None` les noms sont générés ou définis par ailleurs.
- `index_col` noms ou numéros de colonnes définissant les index de lignes, index pouvant être hiérarchisés comme les facteurs d'un plan d'expérience.
- `names` si {\\t header=None}, liste des noms des variables. 
- `nrows` utile pour tester et limiter le nombre de ligne à lire.
- `skiprow` liste de lignes à sauter en lecture.
- `skip_footer` nombre de lignes à sauter en fin de fichier.
- `na_values` définition du ou des codes signalant des valeurs manquantes. Ils peuvent être définis dans un dictionnaire pour associer variables et codes de valeurs manquantes spécifiques.
- `usecols` sélectionne une liste des variable à lire pour éviter de lire des champs ou variables volumineuses et inutiles.
- `skip_blan_lines` à `True` pour sauter les lignes blanches.
- `converters` appliquer une fonction à une colonne ou variable.
- `day_first` par défaut `False`, pour des dates françaises au format `7/06/2013`.
- `chunksize` taille des morceaux à lire itérativement.
- `verbose` imprime des informations comme le nombre de valeurs manquantes des variables non numériques.
- `encoding` type d'encodage comme "utf-8" ou "latin-1"
- `thousand` séparateur des miliers: "." ou ",".

Remarques:
- De nombreuses options de gestion des dates et séries ne sont pas citées.
- `chunksize` provoque la lecture d'un gros fichiers par morceaux de même taille (nombre de lignes). Des fonctions (comptage, dénombrement...) peuvent ensuite s'appliquer itérativement sur les morceaux.


## Exemple
Les données du naufrage du Titanic  illustrent l'utilisation de `pandas`. Elles sont lues directement à partir de leur URL ou sinon les charger [ici](https://e-campus.enac.fr/moodle/course/view.php?id=15036#section-11) vers le répertoire de travail de Python.

In [21]:
# Importations
import pandas as pd
import numpy as np
# tester la lecture
# path=""
path='./'
df = pd.read_csv(path+'titanic-train.csv',nrows=5)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [22]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [24]:
# tout lire
df = pd.read_csv(path+"titanic-train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
df.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [26]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

Vérifions également les valeurs manquantes dans les colonnes qui nous intéressent, et décidons ensuite de ce qu'il faut en faire.

In [27]:
def get_nan_count(column):
    '''
    column - the column for which we want the NaN value count.
    This function returns the number of NaN values in a specific column.
    '''
    nan_count = column.isnull().sum()
    return nan_count

In [28]:
get_nan_count(df['Survived'])

0

In [29]:
get_nan_count(df['Pclass'])

0

In [30]:
get_nan_count(df['Sex'])

0

In [31]:
get_nan_count(df['Age'])

177

In [33]:
get_nan_count(df['Embarked'])

2

Dans l'ensemble, notre analyse ne devra pas prendre les valeurs manquantes pour **la classe de billet** et le **sexe**. Nous ne tiendrons pas compte des valeurs manquantes pour les colonnes **Âge** et **Embarked**.
La colonne **Âge** ayant 177 valeurs nulles pour 891 passagers, les résultats pourraient être moins fiables.

In [34]:
# Des variables sont inexploitables
# Choisir les colonnes utiles
df=pd.read_csv(path+"titanic-train.csv",
    usecols=[1,2,4,5,6,7,9,11],nrows=5)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22,1,0,7.2500,S
1,1,1,female,38,1,0,71.2833,C
2,1,3,female,26,0,0,7.9250,S
3,1,1,female,35,1,0,53.1000,S
4,0,3,male,35,0,0,8.0500,S


À partir de la version 0.15, `pandas`, inclut un type `category` (assez proche de celui ` factor` de R). Il devrait normalement être déclaré dans un dictionnaire au moment par exemple de la lecture (`dtype={"Surv":pd.Categorical...}`) mais ce n'est pas le cas, c'est donc le type objet qui est déclaré puis modifié. Il est vivement recommandé de bien affecter les bons types à chaque variable ne serait-ce que pour éviter de faire des opérations douteuses, par exemple arithmétiques sur des codes de modalités.

In [35]:
df=pd.read_csv(path+"titanic-train.csv",skiprows=1,header=None,usecols=[1,2,4,5,9,11],
  names=["Surv","Classe","Genre","Age","Prix","Port"],dtype={"Surv":object,
    "Classe":object,"Genre":object,"Port":object})
df.head()

,Surv,Classe,Genre,Age,Prix,Port
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S


In [36]:
df.dtypes

Surv       object
Classe     object
Genre      object
Age       float64
Prix      float64
Port       object
dtype: object

Redéfinition des bons types.

In [37]:
df["Surv"]=pd.Categorical(df["Surv"],ordered=False)
df["Classe"]=pd.Categorical(df["Classe"],ordered=False)
df["Genre"]=pd.Categorical(df["Genre"],ordered=False)
df["Port"]=pd.Categorical(df["Port"],ordered=False)
df.dtypes

Surv      category
Classe    category
Genre     category
Age        float64
Prix       float64
Port      category
dtype: object

Remarque : il est également possible de tout lire avant de laisser "tomber" les variable inexploitables. C'est le rôle de la commande: 

`df = df.drop(["Name", "Ticket", "Cabin"], axis=1)`

## Echantillonnage simple
Le type `DataFrame` de Python est chargé en mémoire. Si, malgré les options précédentes permettant de sélectionner, les colonnes, les types des variables... le fichier est encore trop gros, il reste possible, avant de chercher une configuration matérielle lourde et en première approximation, de tirer un échantillon aléatoire simple selon une distribution uniforme. Un tirage stratifié demanderait plus de travail. Cela suppose de connaître le nombre de ligne du fichier ou une valeur inférieure proche.

In [39]:
# pour les données titanic:
N=891  # taille du fichier
n=200  # taille de l'échantillon
lin2skipe=[0] # ne pas lire la première ligne
# ne pas lire N-n lignes tirées aléatoirement
lin2skipe.extend(np.random.choice(np.arange(1,N+1),
    (N-n),replace=False))
df_small=pd.read_csv(path+"titanic-train.csv",
    skiprows=lin2skipe,header=None, 
    usecols=[1,2,4,5,9,11],
    names=["Surv","Classe","Genre","Age",
         "Prix","Port"])
df_small

,Surv,Classe,Genre,Age,Prix,Port
0,1,3,female,26.0,7.9250,S
1,0,1,male,54.0,51.8625,S
2,1,1,female,58.0,26.5500,S
3,0,3,male,39.0,31.2750,S
4,1,2,male,NaN,13.0000,S
...,...,...,...,...,...,...
195,1,2,female,28.0,24.0000,C
196,0,3,male,NaN,7.8958,S
197,1,1,female,56.0,83.1583,C
198,1,2,female,25.0,26.0000,S


# Gérer une table de données


## Discrétisation d'une variable quantitative

Pour la discrétisation d'une variable quantitative. Il est d'un bon usage de définir les bornes des classes à des quantiles, plutôt qu'également espacées, afin de construire des classes d'effectifs sensiblement égales.  Ceci est obtenu par la fonction `qcut`. La fonction `cut` propose par défaut des bornes équi-réparties à moins de fournir une liste de ces bornes.

In [41]:
df["AgeQ"]=pd.qcut(df.Age,3,labels=["Ag1","Ag2",
   "Ag3"])
df["PrixQ"]=pd.qcut(df.Prix,3,labels=["Pr1","Pr2",
   "Pr3"])
df["PrixQ"].describe()

count     891
unique      3
top       Pr1
freq      308
Name: PrixQ, dtype: object

## Modifier / regrouper des modalités
Le recodage des variables qualitatives ou renommage en clair des modalités est obtenu simplement.

In [42]:
df["Surv"]=df["Surv"].cat.rename_categories(
    ["Vnon","Voui"])
df["Classe"]=df["Classe"].cat.rename_categories(
    ["Cl1","Cl2","Cl3"])
df["Genre"]=df["Genre"].cat.rename_categories(
    ["Gfem","Gmas"])
df["Port"]=df["Port"].cat.rename_categories(
    ["Pc","Pq","Ps"])

In [ ]:
df.head()

Il est possible d'associer recodage et regroupement des modalités en définissant un dictionnaire de transformation.

In [ ]:
data = pd.DataFrame({"food":["bacon","pulled pork", 
   "bacon", "Pastrami",
   "corned beef", "Bacon", "pastrami", "honey ham",
   "nova lox"],
   "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}
# Eviter les mélanges de majuscules minuscules 
# en mettant tout en minuscule
data["animal"] = data["food"].map(
    str.lower).map(meat_to_animal)
data

In [ ]:
data["food"].map(lambda x: meat_to_animal[x.lower()])

In [ ]:
dfs = pd.DataFrame({"key": ["b", "b", "a", "c",
    "a", "b"],"data1": range(6)})
pd.get_dummies(dfs["key"])

## Variables indicatrices
Générer des indicatrices des modalités ou *dummy variables*.

In [ ]:
dummies = pd.get_dummies(dfs['key'], prefix='key')
df_with_dummy = dfs[['data1']].join(dummies)
df_with_dummy

## Permutation et tirage aléatoires

Permutation aléatoire :

In [ ]:
dfs = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
sampler = np.random.permutation(5)
sampler
dfs
dfs.take(sampler)

Tirage aléatoire avec remplacement ou *bootstrap* ; celui sans remplacement est traité dans la section dédiée.

In [ ]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)
draws = bag.take(sampler)
draws

## Transformations, opérations
Les opérations arithmétiques entre `Series` et `DataFrame` sont possibles au même titre qu'entre `array`. Si les index ne correspondent pas, des valeurs manquantes (NAN) sont créées à moins d'utiliser des méthodes d'arithmétique `flexible` (`add, sub, div, mul`) autorisant la complétion par une valeur par défaut, généralement 0.

Une fonction quelconque (`lambda`) peut être appliquée avec une même commande qu'`apply` de R. 

In [ ]:
# la table de données
frame = pd.DataFrame(np.random.randn(4,3), 
    columns=list("bde"),
    index=["Utah", "Ohio", "Texas", "Oregon"])
# une fonction
f = lambda x: x.max() - x.min()
frame.apply(f, axis=1)

## Tri et rangs
Trier une table selon les valeurs d'une variable ou d'un index.

In [ ]:
frame = pd.DataFrame(np.arange(8).reshape((2,4)), 
        index=["three", "one"],
        columns=["d", "a", "b", "c"])
frame.sort_index()

In [ ]:
frame.sort_index(axis=1)

In [ ]:
frame.sort_index(axis=1, ascending=False)

In [ ]:
frame.sort_values(by="b")

La commande `rank` remplace les valeurs par leur rang dans l'ordre des lignes ou des colonnes.

In [ ]:
frame = pd.DataFrame({"b": [4.3, 7, -3, 2], 
    "a": [0, 1, 0, 1],"c": [-2, 5, 8, -2.5]})
frame.rank(axis=1)

In [ ]:
frame.rank(axis=0)

# Statistiques descriptives élémentaires
Continuons l'étude des données sur le naufrage du Titanic. Les commandes ci-dessous permettent des premiers diagnostics sur la qualité des données.
## Description univariée

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df["Age"].hist()
plt.show()

In [ ]:
df["Age"].plot(kind="box")
plt.show()

In [ ]:
df["Prix"].plot(kind="hist")
plt.show()

In [ ]:
# qualitatif
df["Surv"].value_counts()

In [ ]:
df["Classe"].value_counts()

In [ ]:
df["Genre"].value_counts()

In [ ]:
df["Port"].value_counts()

## Description bivariée

In [ ]:
df.plot(kind="scatter",x="Age",y="Prix")
plt.show()

In [ ]:
# afficher une sélection
df[df["Age"]>60][["Genre","Classe","Age","Surv"]]

In [ ]:
df.boxplot(column="Age",by="Classe")
plt.show()

In [ ]:
df.boxplot(column="Prix",by="Surv")
plt.show()

In [ ]:
# table de contingence
table=pd.crosstab(df["Surv"],df["Classe"])
print(table)

In [ ]:
# Mosaic plot
from statsmodels.graphics.mosaicplot import mosaic
mosaic(df,["Classe","Genre"])
plt.show()

In [ ]:
mosaic(df,["Surv","Classe"])
plt.show()

## Imputation de données manquantes
La gestion des données manquantes est souvent un point délicat. De nombreuses stratégies ont été élaborées. Nous ne décrivons ici que les plus élémentaires à [mettre en oeuvre](http://pandas.pydata.org/pandas-docs/version/0.15.2/missing_data.html) avec `pandas`.

Il est ainsi facile de supprimer toutes les observations présentant des données manquantes lorsque celles-ci sont peu nombreuses et majoritairement regroupées sur certaines lignes ou colonnes.

``
df = df.dropna(axis=0)
df = df.dropna(axis=1)
``

`Pandas` permet également de faire le choix pour une variable qualitative de considérer `np.nan` comme une modalité spécifique ou d'ignorer l'observation correspondante.

Autres stratégies :
* Cas quantitatif : une valeur manquante est imputée par la moyenne ou la médiane.
* Cas d'une série chronologique : imputation par la valeur précédente ou suivante ou par interpolation linéaire, polynomiale ou encore lissage spline.
* Cas qualitatif : modalité la plus fréquente ou répartition aléatoire selon les fréquences observées des modalités.

La variable âge contient de nombreuses données manquantes. La fonction `fillna` présente plusieurs options d'imputation.

In [ ]:
# Remplacement par la médiane d'une variable quantitative
#df=df.fillna(df.median())
df.describe()

In [ ]:
# par la modalité "médiane" de AgeQ
df.info()
df.AgeQ=df["AgeQ"].fillna("Ag2")
# par le port le plus fréquent
df["Port"].value_counts()
df.Port=df["Port"].fillna("Ps")
df.info()

Ces imputations sont assez rudimentaires et d'autres sont à privilégier pour des modélisations plus soignées mais ces méthodes font appel généralement à d'autres outils.

D'autres fonctions (Mac Kinney, 2013) sont proposées pour supprimer les duplicatas (` drop\_duplicates`), modifier les dimensions, traquer des anomalies unidimensionnelles, selon un modèle gaussien ou par rapport à des quantiles.

# Manipuler des tables de données

## Jointure
Il s'agit de "jointer" deux tables partageant la même clef ou encore de concaténer horizontalement les lignes en faisant correspondre les valeurs d'une variable clef qui peuvent ne pas être uniques.

In [ ]:
# tables
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", 
     "a","a", "b"],"data1": range(7)})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
    "data2": range(3)})
pd.merge(df1,df2,on="key")

La gestion des clefs manquantes est en option : entre autres, ne pas introduire de ligne (ci-dessus), insérer des valeurs manquantes ci-dessous. 

In [ ]:
# valeurs manquantes
pd.merge(df1,df2,on="key", how="outer")

## Concaténation selon un axe
Concaténation verticale (axis=0) ou horizontales (axis=1) de tables. La concaténation horizontale est similaire à la jointure (option `outer`).

In [ ]:
# tables
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", 
    "a", "a", "b"],"var": range(7)})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
    "var": range(3)})
# concaténation verticales
pd.concat([df1,df2],axis=0)

In [ ]:
# concaténation horizontale
pd.concat([df1,df2],axis=1)

# Manipulations séquentielles de gros fichiers

Étape suivante associée à la croissance du volume : les fichiers des données brutes ne tiennent pas en mémoire.  Il "suffit" d'intégrer ou enchâsser les étapes des sections précédentes dans la lecture séquentielle d'un gros fichier. En apparence simple d'un point de vue méthodologique, cette étape peut consommer beaucoup de temps par tests et remises en causes incessantes des choix de sélection, transformation, recodage... des variables. Il est crucial de se doter d'outils efficaces. 

Il s'agit donc de lire les données par morceau (nombre fixé de lignes) ou ligne à ligne, traiter chaque morceau, le ré-écrire dans un fichier de format binaire plutôt que texte ; le choix du format HDF5 semble le plus efficace du point de vue technique et pour servir d'interface à d'autres environnements : C, java, Matlab... et R car une librairie (rhdf5 de Bioconductor) gère ce format.

La procédure est comparable à une étape `Data` de SAS, qui lit/écrit les tables ligne à ligne.

Deux librairies: `h5py` et `PyTables` gèrent le format HDF5 en Python. Pour simplifier la tâche, `pandas` intègre une classe `HDFStore` utilisant `PyTables` qui doit donc être installée. 

**Attention**: ce format n'est pas adapté à une gestion *parallélisée*, notamment en écriture. 


## Lecture séquentielle

L'exemple est ici donné pour lire un fichier texte mais beaucoup d'autres formats (excel, hdf, sql, json, msgpack, html, gbq, stata, clipboard, pickle) sont connus de `pandas`.

In [ ]:
# importations
import pandas as pd
import numpy as np
# lire tout le fichier par morceaux
# avec l'option chunksize
Partition=pd.read_csv(path+"titanic-train.csv",skiprows=1,
  header=None,usecols=[1,2,4,5,9,11],
  names=["Surv","Classe","Genre","Age",
    "Prix","Port"],dtype={"Surv":object,
    "Classe":object,"Genre":object,"Port":object},
    chunksize=100)
# ouverture du fichier HDF5
stock=pd.HDFStore("titan.h5")
# boucle de lecture
for Part in Partition:
    # "nettoyage" préliminaire des données
    #Part=Part.drop(["Name","Ticket","Cabin"],axis=1)
    # ... autres opérations
    # création de la table "df" dans "stock" puis
    # extension de celle-ci par chaque "Part"
    stock.append("df",Part)
# dernier morceau lu et ajouté
Part.head()

In [ ]:
# Il est généralement utile de fermer le fichier
stock.close()

**Attention** aux types implicites des variables. Si, par exemple, une donnée manquante n'apparaît pas dans une colonne du 1er morceau mais dans le 2ème, cela peut engendrer un conflit de type. Expliciter systématiquement les types et noms des variables dans un dictionnaire en paramètre.

## Utilisation d'une table HDF5

In [ ]:
# Ouverture du fichier
Archiv=pd.HDFStore("titan.h5")
# sélection de la table et affichage de l'entête
Archiv.select("df").head()

Cette partie est à développer pour illustrer les fonctionnalités de `pandas` permettant d'interroger / requêter (*querying* notamment SQL) une table archivée dans un fichier HDF5. Consulter la [documentation en ligne](http://pandas.pydata.org/pandas-docs/dev/io.html#hdf5-pytables) à ce sujet. 

## Echantillon aléatoire simple
Le fichier créé au format HDF5 peut être encore très volumineux. Par souci d'efficacité, son raffinement, son exploitation, voire même son analyse pour modélisation, peuvent ou même doivent être opérés sur un  simple échantillon aléatoire.

In [ ]:
# extraction du nombre de lignes  / individus
nrows = Archiv.get_storer("df").nrows
# génération des index aléatoires
r = np.random.randint(0,nrows,size=10)
print(r)

In [ ]:
# extraction des lignes d'index fixés
df_ech=Archiv.select("df",where=pd.Index(r))
df_ech

Il "suffit" alors d'appliquer les outils des sections précédentes.

## À suivre...
Ces traitements font appel à de très nombreuses opérations de lectures / écritures sur un seul ordinateur, un seul disque au regard du volume des calculs ; ils ne sont pas adaptés à une parallélisation sur un ordinateur multiprocesseur. La gestion et l'analyse de plus gros volumes de données nécessite une distribution de celles-ci sur plusieurs serveurs / disques. D'autres technologies doivent être utilisées ; c'est actuellement le couple *Spark/Hadoop* le plus en vogue.

**Intérêt** : *Spark* est utilisable avec java, Scala et aussi Python. L'investissement dans ce langage est donc rentable. 

<hr>

## Références

**Mac Kinney W.** (2013). *Python for Data Analysis*, O’Reilly. [pdf](http://it-ebooks.info/book/104)